In [3]:
import pandas as pd
file = r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\heba.csv'
df = pd.read_csv(file)
df = df[df['Description'].fillna('').str.strip() != ""]
df = df.dropna()
df

,Product,Variant,Price,Handle,Variant ID,Availability,SKU,Product URL,Scraped at,Description
0,Calligraphy - Duvet set,King / White,14495.0,calligraphy-duvet-set,46336742719779,False,672b94526d860,https://hebalinens.com/products/calligraphy-du...,2026/02/01 03:57:54 PM,A beautiful tribute to the legendary singer Om...
1,Calligraphy - Bath Mat Set,White,4295.0,calligraphy-bath-mat-set,46635615027491,False,672b9451e4a21,https://hebalinens.com/products/calligraphy-ba...,2026/02/01 03:57:56 PM,Step onto the luxurious comfort of this Callig...
2,Astronauts - Swaddle Towel sets,White,995.0,astronauts-swaddle-towel-sets,46383258698019,False,672b9440b801c,https://hebalinens.com/products/astronauts-swa...,2026/02/01 03:57:58 PM,Snuggle your little space explorer with our As...
3,Astronauts - Swaddle Towel sets,Cream,995.0,astronauts-swaddle-towel-sets,46383258763555,True,672b943fec8b4,https://hebalinens.com/products/astronauts-swa...,2026/02/01 03:57:58 PM,Snuggle your little space explorer with our As...
4,Flow - Towel Set,White,2195.0,flow-towels-set,46867430703395,False,672b95f9ce279,https://hebalinens.com/products/flow-towels-set,2026/02/01 03:58:01 PM,Experience the best of both worlds—the luxurio...
...,...,...,...,...,...,...,...,...,...,...
77,Farashat - Guest Towel Set,White,480.0,farashat-guest-towel-set,47052814745891,True,672b95cb63556,https://hebalinens.com/products/farashat-guest...,2026/02/01 03:59:16 PM,Introducing Farashat's luxurious guest towel s...
78,Farashat - Guest Towel Set,Cream,480.0,farashat-guest-towel-set,47067664351523,True,672b95ca8f539,https://hebalinens.com/products/farashat-guest...,2026/02/01 03:59:16 PM,Introducing Farashat's luxurious guest towel s...
79,Farashat - Towel Set,White,1795.0,farashat-towels-set-3pcs,45686430630179,False,672b95cedc178,https://hebalinens.com/products/farashat-towel...,2026/02/01 03:59:17 PM,Wrap yourself in luxury with Farashat's exquis...
80,Farashat - Towel Set,Cream,1795.0,farashat-towels-set-3pcs,45686430662947,True,672b95ce287db,https://hebalinens.com/products/farashat-towel...,2026/02/01 03:59:17 PM,Wrap yourself in luxury with Farashat's exquis...


In [4]:
# import pandas as pd

# # Define how to handle each column during the merge
# aggregation_rules = {
#     'Variant': lambda x: list(x.unique()), # Create a "White, Cream, Blue" string
#     'Variant ID': lambda x: list(x.astype(str).unique()),               # Create a list of IDs
#     'SKU': lambda x: list(x.unique()),                      # Create a list of SKUs
#     'Handle': 'first',                                      # Keep the first value found
#     'Availability': 'first',
#     'Product URL': 'first',
#     'Scraped at': 'first'
# }

# # Group by the "Core" identity of the product and apply the rules
# df_merged = df.groupby(['Product', 'Description', 'Price'], as_index=False).agg(aggregation_rules)

# # Reorder columns to match your original header order
# columns_order = [
#     'Product', 'Variant', 'Price', 'Handle', 'Variant ID', 
#     'Availability', 'SKU', 'Product URL', 'Scraped at', 
#     'Description'
# ]
# df_merged = df_merged[columns_order]
# df_merged['Hollistic Description'] = df_merged['Product'] + ' - ' + str(df_merged['Variant']) + ": " + df_merged["Description"]
# df_merged.to_csv(r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\heba_variants_cleaned.csv')

import pandas as pd

# 1. Clean the 'Description' column BEFORE merging to remove new lines
df['Description'] = df['Description'].str.replace(r'\n|\r', ' ', regex=True)

# 2. Updated aggregation rules to create clean strings instead of Python lists
# We use ', '.join() to make "White, Cream" instead of "['White', 'Cream']"
aggregation_rules = {
    'Variant': lambda x: ', '.join(map(str, x.unique())), 
    'Variant ID': lambda x: ', '.join(map(str, x.unique())),
    'SKU': lambda x: ', '.join(map(str, x.unique())),
    'Handle': 'first',
    'Availability': 'first',
    'Product URL': 'first',
    'Scraped at': 'first'
}

# Group by the "Core" identity
df_merged = df.groupby(['Product', 'Description', 'Price'], as_index=False).agg(aggregation_rules)

# 3. FIX: Create the Hollistic Description correctly
# Use .astype(str) instead of str() to handle it row-by-row
df_merged['Hollistic Description'] = (
    df_merged['Product'] + 
    ' - ' + 
    df_merged['Variant'].astype(str) + 
    ": " + 
    df_merged["Description"]
)

# Reorder columns
columns_order = [
    'Product', 'Variant', 'Price', 'Handle', 'Variant ID', 
    'Availability', 'SKU', 'Product URL', 'Scraped at', 
    'Description', 'Hollistic Description'
]
df_merged = df_merged[columns_order]

# Save to CSV (index=False prevents that extra "0, 1, 2" column at the start)
df_merged.to_csv(r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\heba_compact_view.csv', index=False)
df_merged

,Product,Variant,Price,Handle,Variant ID,Availability,SKU,Product URL,Scraped at,Description,Hollistic Description
0,Astronauts - Swaddle Towel sets,"White, Cream",995.0,astronauts-swaddle-towel-sets,"46383258698019, 46383258763555",False,"672b9440b801c, 672b943fec8b4",https://hebalinens.com/products/astronauts-swa...,2026/02/01 03:57:58 PM,Snuggle your little space explorer with our As...,"Astronauts - Swaddle Towel sets - White, Cream..."
1,Baby animals - Swaddle Towel sets,"White, Cream",995.0,baby-animals-swaddle-towel-sets,"46383232647459, 46383232712995",False,"672b9440b0b7a, 672b943fe31db",https://hebalinens.com/products/baby-animals-s...,2026/02/01 03:58:17 PM,"Soft and cuddly, this baby swaddle towel set w...","Baby animals - Swaddle Towel sets - White, Cre..."
2,Ballerina - Bath robe,4 years to 6 years / White,645.0,ballerina-bath-robe,46381868187939,True,672b943feffbe,https://hebalinens.com/products/ballerina-bath...,2026/02/01 03:58:23 PM,"Introducing Ballerina, the perfect 100% cotton...",Ballerina - Bath robe - 4 years to 6 years / W...
3,Ballerina - Bath robe,6 years to 8 years / White,945.0,ballerina-bath-robe,46381868253475,False,672b9440ad92d,https://hebalinens.com/products/ballerina-bath...,2026/02/01 03:58:23 PM,"Introducing Ballerina, the perfect 100% cotton...",Ballerina - Bath robe - 6 years to 8 years / W...
4,Ballerina - Duvet set,White,6395.0,ballerina-duvet-set,46381754384675,True,672b9442c140d,https://hebalinens.com/products/ballerina-duve...,2026/02/01 03:58:08 PM,Let your dreams take flight with this Ballerin...,Ballerina - Duvet set - White: Let your dreams...
5,Boxer Underwear bag,White / 60 L X 40 W,395.0,boxer-underwear-bag,46353713201443,True,672b944ebfea1,https://hebalinens.com/products/boxer-underwea...,2026/02/01 03:58:15 PM,Make your travels stylish and hassle-free with...,Boxer Underwear bag - White / 60 L X 40 W: Mak...
6,Bra and Underwear bag,White / 60 L X 40 W,395.0,bra-and-underwear-bag,46353877205283,True,672b944f4d8e4,https://hebalinens.com/products/bra-and-underw...,2026/02/01 03:58:05 PM,Pack and travel in style with this designer Br...,Bra and Underwear bag - White / 60 L X 40 W: P...
7,Butterflies - Table Cloth,2.75 L x 1.8 W with 8 Napkin / Cream,6245.0,table-cloth-batterflies,46344867905827,True,672b9451e57ce,https://hebalinens.com/products/table-cloth-ba...,2026/02/01 03:58:20 PM,Brighten up your dining experience with the ey...,Butterflies - Table Cloth - 2.75 L x 1.8 W wi...
8,Butterflies - Table Cloth,3.25 L x 1.8 W with 10 Napkin / Cream,7495.0,table-cloth-batterflies,46344867971363,False,672b9452b1858,https://hebalinens.com/products/table-cloth-ba...,2026/02/01 03:58:20 PM,Brighten up your dining experience with the ey...,Butterflies - Table Cloth - 3.25 L x 1.8 W wit...
9,Calligraphy - Bath Mat Set,White,4295.0,calligraphy-bath-mat-set,46635615027491,False,672b9451e4a21,https://hebalinens.com/products/calligraphy-ba...,2026/02/01 03:57:56 PM,Step onto the luxurious comfort of this Callig...,Calligraphy - Bath Mat Set - White: Step onto ...


In [5]:
'''
Products Table
'''
df_products = df_merged
df_products = df_products.drop(columns=['Variant', 'Price', 'Variant ID', 'Availability', 'SKU'])
df_products.to_csv(r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\heba_products.csv', index=False)
df_products

,Product,Handle,Product URL,Scraped at,Description,Hollistic Description
0,Astronauts - Swaddle Towel sets,astronauts-swaddle-towel-sets,https://hebalinens.com/products/astronauts-swa...,2026/02/01 03:57:58 PM,Snuggle your little space explorer with our As...,"Astronauts - Swaddle Towel sets - White, Cream..."
1,Baby animals - Swaddle Towel sets,baby-animals-swaddle-towel-sets,https://hebalinens.com/products/baby-animals-s...,2026/02/01 03:58:17 PM,"Soft and cuddly, this baby swaddle towel set w...","Baby animals - Swaddle Towel sets - White, Cre..."
2,Ballerina - Bath robe,ballerina-bath-robe,https://hebalinens.com/products/ballerina-bath...,2026/02/01 03:58:23 PM,"Introducing Ballerina, the perfect 100% cotton...",Ballerina - Bath robe - 4 years to 6 years / W...
3,Ballerina - Bath robe,ballerina-bath-robe,https://hebalinens.com/products/ballerina-bath...,2026/02/01 03:58:23 PM,"Introducing Ballerina, the perfect 100% cotton...",Ballerina - Bath robe - 6 years to 8 years / W...
4,Ballerina - Duvet set,ballerina-duvet-set,https://hebalinens.com/products/ballerina-duve...,2026/02/01 03:58:08 PM,Let your dreams take flight with this Ballerin...,Ballerina - Duvet set - White: Let your dreams...
5,Boxer Underwear bag,boxer-underwear-bag,https://hebalinens.com/products/boxer-underwea...,2026/02/01 03:58:15 PM,Make your travels stylish and hassle-free with...,Boxer Underwear bag - White / 60 L X 40 W: Mak...
6,Bra and Underwear bag,bra-and-underwear-bag,https://hebalinens.com/products/bra-and-underw...,2026/02/01 03:58:05 PM,Pack and travel in style with this designer Br...,Bra and Underwear bag - White / 60 L X 40 W: P...
7,Butterflies - Table Cloth,table-cloth-batterflies,https://hebalinens.com/products/table-cloth-ba...,2026/02/01 03:58:20 PM,Brighten up your dining experience with the ey...,Butterflies - Table Cloth - 2.75 L x 1.8 W wi...
8,Butterflies - Table Cloth,table-cloth-batterflies,https://hebalinens.com/products/table-cloth-ba...,2026/02/01 03:58:20 PM,Brighten up your dining experience with the ey...,Butterflies - Table Cloth - 3.25 L x 1.8 W wit...
9,Calligraphy - Bath Mat Set,calligraphy-bath-mat-set,https://hebalinens.com/products/calligraphy-ba...,2026/02/01 03:57:56 PM,Step onto the luxurious comfort of this Callig...,Calligraphy - Bath Mat Set - White: Step onto ...


In [6]:
'''
Variants Table
'''

df_variants = df
df_variants = df_variants.drop(columns=['Product', "Product URL", 'Description'])
cols_to_move = ['Variant ID', 'Handle', 'Variant', 'Price', 'SKU', 'Scraped at']

# Get the list of all other columns that aren't in the "move" list
remaining_cols = [c for c in df_variants.columns if c not in cols_to_move]

# Reorder with move-list first, then the rest
df_variants = df[cols_to_move + remaining_cols]
df_variants.to_csv(r'C:\Users\moham\OneDrive\Desktop\WebScraping\legacy_app\heba_variants.csv', index=False)
df_variants

,Variant ID,Handle,Variant,Price,SKU,Scraped at,Availability
0,46336742719779,calligraphy-duvet-set,King / White,14495.0,672b94526d860,2026/02/01 03:57:54 PM,False
1,46635615027491,calligraphy-bath-mat-set,White,4295.0,672b9451e4a21,2026/02/01 03:57:56 PM,False
2,46383258698019,astronauts-swaddle-towel-sets,White,995.0,672b9440b801c,2026/02/01 03:57:58 PM,False
3,46383258763555,astronauts-swaddle-towel-sets,Cream,995.0,672b943fec8b4,2026/02/01 03:57:58 PM,True
4,46867430703395,flow-towels-set,White,2195.0,672b95f9ce279,2026/02/01 03:58:01 PM,False
...,...,...,...,...,...,...,...
77,47052814745891,farashat-guest-towel-set,White,480.0,672b95cb63556,2026/02/01 03:59:16 PM,True
78,47067664351523,farashat-guest-towel-set,Cream,480.0,672b95ca8f539,2026/02/01 03:59:16 PM,True
79,45686430630179,farashat-towels-set-3pcs,White,1795.0,672b95cedc178,2026/02/01 03:59:17 PM,False
80,45686430662947,farashat-towels-set-3pcs,Cream,1795.0,672b95ce287db,2026/02/01 03:59:17 PM,True
